In [15]:
#dl.ipynb
#
#by Joe Hahn
#jmh.datasciences@gmail.com
#24 January 2018

In [49]:
#read list of documents...each 'document' is a list of sentences from each book's middle 70%
import subprocess, pickle
files = subprocess.check_output(['find', 'data', '-name', '*.pkl']).decode('utf8').split('\n')[0:-1]
docs = []
for file in files:
    with open(file, 'rb') as fp:
        doc = [pickle.load(fp)][0]
        docs += [doc]
print 'number of documents read = ', len(docs)

number of documents read =  192


In [54]:
#import matplotlib pandas etc
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd
from matplotlib import rcParams
sns.set(font_scale=1.5, font='DejaVu Sans')
pd.set_option('display.max_columns', None)

In [62]:
meta_data = [doc[-1] for doc in docs]
cols = ['author', 'title', 'N_sentences', 'input_file']
docs_df = pd.DataFrame(meta_data)[cols].sort_values('author')
docs_df.head()

,author,title,N_sentences,input_file
150,A. Conan Doyle,The Sign of the Four,2408,iso/etext00/sign410.txt
97,A.H. Clough,PlutarchLives,13656,iso/etext96/plivs10.txt
161,Abraham Lincoln,"The Writings of Lincoln, v7",3094,iso/etext01/7linc11.txt
123,Adolphus William Ward,Chaucer,1765,iso/etext03/chacr10.txt
166,Agatha Christie,Secret Adversary,6518,iso/etext98/secad10.txt


In [73]:
doc_list = []
for doc in docs:
    meta = doc[-1]
    sentences = doc[:-1]
    meta['sentences'] = sentences
    doc_list += [meta]
cols = ['author', 'title', 'N_sentences', 'sentences', 'input_file']
docs_df = pd.DataFrame(doc_list)[cols].sort_values('author')

In [74]:
docs_df.head()

,author,title,N_sentences,sentences,input_file
150,A. Conan Doyle,The Sign of the Four,2408,"[""She did not think so., But at least you cann...",iso/etext00/sign410.txt
97,A.H. Clough,PlutarchLives,13656,"[From thence he fled into Epirus, and,\r\nthe ...",iso/etext96/plivs10.txt
161,Abraham Lincoln,"The Writings of Lincoln, v7",3094,"[Her Britannic Majesty's government, as was ju...",iso/etext01/7linc11.txt
123,Adolphus William Ward,Chaucer,1765,"[When, upon the declaration of the Papal Schis...",iso/etext03/chacr10.txt
166,Agatha Christie,Secret Adversary,6518,"[""Well, anyway, what ARE we going to\r\ndo?, W...",iso/etext98/secad10.txt


In [65]:
doc[-6:]

["_Damn<e'> et Calive_, 49.",
 "Darius, 54\r\nDekker's _Satiro-Mastix_, errata to, 80.",
 'Deleted _for_ delited in Shakespeare, 115.',
 'De Morgan, on authors correcting their own\r\n     proofs, 89.',
 "D'Israeli's _Curiosities of Literature_, 68, 69.",
 {'N_sentences': 1668,
  'author': 'Henry Wheatley',
  'input_file': u'iso/etext95/litbl10.txt',
  'title': 'Literary Blunders'}]

In [44]:
len(d)

1

In [48]:
d[0][-5:]

['SALISBURY.',
 "I did not think the king so stor'd with friends.",
 'PEMBROKE.',
 'Up once again; put spirit in the French;\r\nIf they miscarry, we miscarry too.',
 {'N_sentences': 1412,
  'author': 'Shakespeare',
  'input_file': u'iso/etext98/2ws1410.txt',
  'title': 'King John'}]